<a href="https://colab.research.google.com/github/glee255/jupiter-notebooks/blob/main/Fetch_HomeTest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Brands Table**
 - Read json file.
 - Read id values within the dictionaries.

In [33]:
# Read json file
import pandas as pd
import json

with open('/content/brands.json', 'r') as file:
    json_text = file.read()

json_objects = json_text.strip().split('\n')
data = [json.loads(obj) for obj in json_objects]
brands = pd.DataFrame(data)

In [34]:
# Read  id values
def extract(d):
    try:
        if "$oid" in d:
            return d["$oid"]
        elif "$id" in d and "$oid" in d["$id"]:
            return d["$id"]["$oid"]
        else:
            return None
    except (KeyError, TypeError):
        return None

brands['brand_id'] = brands['_id'].apply(lambda x: extract(x))
brands['cpg_id'] = brands['cpg'].apply(lambda x: extract(x))

In [35]:
# 1167 brands in total
brands.shape

(1167, 10)

### Missing values
 - There seems to be too many missing values in category, categoryCode, topBrand, brandCode.
 - Removing null values may remove the entire data set. Replacing null values does not make sense for this case.

In [37]:
brands.isna().sum()

_id               0
barcode           0
category        155
categoryCode    650
cpg               0
name              0
topBrand        612
brandCode       234
brand_id          0
cpg_id            0
dtype: int64

### Misleading column name
 - categoryCode seems to have text values and not categorical values.
 - Also it almost seems same as the category values.

In [38]:
brands['categoryCode'].value_counts()

BAKING                           359
CANDY_AND_SWEETS                  71
BEER_WINE_SPIRITS                 31
HEALTHY_AND_WELLNESS              14
GROCERY                           11
BABY                               7
CLEANING_AND_HOME_IMPROVEMENT      6
BREAD_AND_BAKERY                   5
DAIRY_AND_REFRIGERATED             5
PERSONAL_CARE                      4
BEVERAGES                          1
OUTDOOR                            1
MAGAZINES                          1
FROZEN                             1
Name: categoryCode, dtype: int64

In [39]:
brands['category'].value_counts()

Baking                         369
Beer Wine Spirits               90
Snacks                          75
Candy & Sweets                  71
Beverages                       63
Magazines                       44
Health & Wellness               44
Breakfast & Cereal              40
Grocery                         39
Dairy                           33
Condiments & Sauces             27
Frozen                          24
Personal Care                   20
Baby                            18
Canned Goods & Soups            12
Beauty                           9
Cleaning & Home Improvement      6
Deli                             6
Beauty & Personal Care           6
Household                        5
Bread & Bakery                   5
Dairy & Refrigerated             5
Outdoor                          1
Name: category, dtype: int64

### Inconsistent values
 - brandCode contains inconsistent values such as numeric, text and numeric and text combined.

In [40]:
brands['brandCode'].value_counts()

                                 35
GOODNITES                         2
HUGGIES                           2
ROYAL DANSK                       1
SOL                               1
                                 ..
TEST BRANDCODE @1599159969028     1
TEST BRANDCODE @1597350074404     1
SEDAL                             1
RED ROCK DELI                     1
TEST BRANDCODE @1613158231644     1
Name: brandCode, Length: 897, dtype: int64

### Drop irrelevant columns
 - Remove '_id', 'cpg', 'categoryCode columns.
 - 'cpg' value needs to be checked.

In [41]:
# Remove '_id','cpg','categoryCode. 'cpg' value needs to be checked.
colsToDrop = ['_id','categoryCode','cpg']
brands.drop(colsToDrop, axis=1, inplace=True)

### Check duplicates
- No duplicates

In [42]:
# No duplicates
brands.duplicated().any()

False

In [ ]:
# Check the first five rows
brands.head()

### Sqlite3 table conversion

In [44]:
file_name = 'brands.xlsx'
brands.to_excel(file_name, index=False)

In [45]:
for column in brands.select_dtypes(include=['float64']).columns:
    brands[column] = brands[column].fillna(0).astype(int)

for column in brands.select_dtypes(include=['object']).columns:
    brands[column] = brands[column].astype(str)

In [46]:
import sqlite3

conn = sqlite3.connect('fetch.db')
cur = conn.cursor()

brands.to_sql('brands', conn, if_exists='replace', index=False)

1167

### **Receipts Table**
 - Read json file.
 - Retrieve the dictionary values from rewardsReceiptItemList column. Twelve dictionaries contained.
 - Retrieve id values within the dictionary.
 - Retrieve and convert dates from date columns

In [173]:
# Read json file
with open('/content/receipts.json', 'r') as file:
    json_text = file.read()

json_objects = json_text.strip().split('\n')
data1 = [json.loads(obj) for obj in json_objects]
receipts = pd.DataFrame(data1)

In [174]:
# Retrieve the dictionaries from rewardsReceiptItemList column. Too many dictionaries contained.
def extract1(list_of_dicts, key):
    if not isinstance(list_of_dicts, list):
        return ''
    values = [str(d.get(key)) for d in list_of_dicts if d.get(key) is not None]
    return ', '.join(values)

receipts['barcodes'] = receipts['rewardsReceiptItemList'].apply(lambda x: extract1(x, 'barcode'))
receipts['description'] = receipts['rewardsReceiptItemList'].apply(lambda x: extract1(x, 'description'))
receipts['finalPrice'] = receipts['rewardsReceiptItemList'].apply(lambda x: extract1(x, 'finalPrice'))
receipts['itemPrice'] = receipts['rewardsReceiptItemList'].apply(lambda x: extract1(x, 'itemPrice'))
receipts['needsFetchReview'] = receipts['rewardsReceiptItemList'].apply(lambda x: extract1(x, 'needsFetchReview'))
receipts['partnerItemId'] = receipts['rewardsReceiptItemList'].apply(lambda x: extract1(x, 'partnerItemId'))
receipts['preventTargetGapPoints'] = receipts['rewardsReceiptItemList'].apply(lambda x: extract1(x, 'preventTargetGapPoints'))
receipts['quantityPurchased'] = receipts['rewardsReceiptItemList'].apply(lambda x: extract1(x, 'quantityPurchased'))
receipts['userFlaggedBarcode'] = receipts['rewardsReceiptItemList'].apply(lambda x: extract1(x, 'userFlaggedBarcode'))
receipts['userFlaggedNewItem'] = receipts['rewardsReceiptItemList'].apply(lambda x: extract1(x, 'userFlaggedNewItem'))
receipts['userFlaggedPrice'] = receipts['rewardsReceiptItemList'].apply(lambda x: extract1(x, 'userFlaggedPrice'))
receipts['userFlaggedQuantity'] = receipts['rewardsReceiptItemList'].apply(lambda x: extract1(x, 'userFlaggedQuantity'))
receipts['receipt_id'] = receipts['_id'].apply(lambda x: x.get('$oid') if isinstance(x, dict) else None)

In [175]:
# Retrieve dates from date columns
def extract2(d):
    try:
        oid_value = d['$date']
        return oid_value
    except (KeyError, TypeError):
        return None

receipts['create_d'] = receipts['createDate'].apply(lambda x: extract2(x))
receipts['scan_d'] = receipts['dateScanned'].apply(lambda x: extract2(x))
receipts['finish_d'] = receipts['finishedDate'].apply(lambda x: extract2(x))
receipts['modify_d'] = receipts['modifyDate'].apply(lambda x: extract2(x))
receipts['point_d'] = receipts['pointsAwardedDate'].apply(lambda x: extract2(x))
receipts['purchase_d'] = receipts['purchaseDate'].apply(lambda x: extract2(x))

In [176]:
# Convert data to year and month
from datetime import datetime
receipts['create_ym'] = receipts['create_d'].apply(lambda x: datetime.fromtimestamp(x/1000).strftime('%Y-%m') if pd.notnull(x) else None)
receipts['scan_ym'] = receipts['scan_d'].apply(lambda x: datetime.fromtimestamp(x/1000).strftime('%Y-%m') if pd.notnull(x) else None)
receipts['finish_ym'] = receipts['finish_d'].apply(lambda x: datetime.fromtimestamp(x/1000).strftime('%Y-%m') if pd.notnull(x) else None)
receipts['modify_ym'] = receipts['modify_d'].apply(lambda x: datetime.fromtimestamp(x/1000).strftime('%Y-%m') if pd.notnull(x) else None)
receipts['point_ym'] = receipts['point_d'].apply(lambda x: datetime.fromtimestamp(x/1000).strftime('%Y-%m') if pd.notnull(x) else None)
receipts['purchase_ym'] = receipts['purchase_d'].apply(lambda x: datetime.fromtimestamp(x/1000).strftime('%Y-%m') if pd.notnull(x) else None)

### Drop irrelevant columns

- Remove json formatted columns and columns that were used for retrieving and conversion.



In [177]:
# Remove '_id','createDate','dateScanned','finishedDate','modifyDate','pointsAwardedDate','purchaseDate'
colsToDrop = ['rewardsReceiptItemList','_id','create_d','scan_d','finish_d','modify_d','point_d','purchase_d','createDate','dateScanned','finishedDate','modifyDate','pointsAwardedDate','purchaseDate']
receipts.drop(colsToDrop, axis=1, inplace=True)

### Missing values
 - Customers may not earn bonus points. Null value reasonable.
 - But other than the bonus points related columns, other needs to be checked.
 - Barcodes column contatins empty string that is not recognized as na value.

In [178]:
receipts.isna().sum()

bonusPointsEarned          575
bonusPointsEarnedReason    575
pointsEarned               510
purchasedItemCount         484
rewardsReceiptStatus         0
totalSpent                 435
userId                       0
barcodes                     0
description                  0
finalPrice                   0
itemPrice                    0
needsFetchReview             0
partnerItemId                0
preventTargetGapPoints       0
quantityPurchased            0
userFlaggedBarcode           0
userFlaggedNewItem           0
userFlaggedPrice             0
userFlaggedQuantity          0
receipt_id                   0
create_ym                    0
scan_ym                      0
finish_ym                  551
modify_ym                    0
point_ym                   582
purchase_ym                448
dtype: int64

In [179]:
# empty string not recognized as na value
receipts['barcodes'].head()

0                  4011
1    4011, 028400642255
2                      
3                  4011
4            4011, 1234
Name: barcodes, dtype: object

### Data issues
 - Multiple values in barcodes, finalPrice, itemPrice, partnerItemId, quantityPurchased columns. First form normalization needed.
 - Split the barcodes to use as foreign key.
 - pointsEarned, totalSpent, finalPrice, itemPrice, quantityPurchased data types are object. Data type need to be changed to numeric.

In [180]:
import numpy as np
receipts['barcodes'] = receipts['barcodes'].replace(r'^\s*$', np.nan, regex=True)

In [181]:
receipts['barcodes'] = receipts['barcodes'].str.split(',')
receipts = receipts.explode('barcodes')
receipts.rename(columns={'barcodes': 'barcode'}, inplace=True)
receipts['barcode'] = receipts['barcode'].str.strip()

In [182]:
to_convert = ['pointsEarned', 'totalSpent', 'finalPrice', 'itemPrice', 'quantityPurchased']

# Convert each column to numeric, coercing errors to NaN
for column in to_convert:
    receipts[column] = pd.to_numeric(receipts[column], errors='coerce')

In [183]:
receipts.shape

(3616, 26)

In [184]:
receipts.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3616 entries, 0 to 1118
Data columns (total 26 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   bonusPointsEarned        2709 non-null   float64
 1   bonusPointsEarnedReason  2709 non-null   object 
 2   pointsEarned             2972 non-null   float64
 3   purchasedItemCount       3132 non-null   float64
 4   rewardsReceiptStatus     3616 non-null   object 
 5   totalSpent               3181 non-null   float64
 6   userId                   3616 non-null   object 
 7   barcode                  3090 non-null   object 
 8   description              3616 non-null   object 
 9   finalPrice               433 non-null    float64
 10  itemPrice                433 non-null    float64
 11  needsFetchReview         3616 non-null   object 
 12  partnerItemId            3616 non-null   object 
 13  preventTargetGapPoints   3616 non-null   object 
 14  quantityPurchased       

In [185]:
receipts.isna().sum()

bonusPointsEarned           907
bonusPointsEarnedReason     907
pointsEarned                644
purchasedItemCount          484
rewardsReceiptStatus          0
totalSpent                  435
userId                        0
barcode                     526
description                   0
finalPrice                 3183
itemPrice                  3183
needsFetchReview              0
partnerItemId                 0
preventTargetGapPoints        0
quantityPurchased          3183
userFlaggedBarcode            0
userFlaggedNewItem            0
userFlaggedPrice              0
userFlaggedQuantity           0
receipt_id                    0
create_ym                     0
scan_ym                       0
finish_ym                   797
modify_ym                     0
point_ym                    811
purchase_ym                 448
dtype: int64

### Sqlite3 table conversion

In [186]:
for column in receipts.select_dtypes(include=['float64']).columns:
    receipts[column] = receipts[column].fillna(0).astype(int)

In [187]:
receipts.to_sql('receipts', conn, if_exists='replace', index=False)

3616

In [188]:
file_name = 'receipts.xlsx'
receipts.to_excel(file_name, index=False)

### **Users Table**
 - Read json file.
 - Retrieve id values within the oid dictionary.
 - Retrieve and convert dates from date dictionary.

In [223]:
with open('/content/users.json', 'r') as file:
    json_text = file.read()

json_objects = json_text.strip().split('\n')
data2 = [json.loads(obj) for obj in json_objects]
users = pd.DataFrame(data2)

users.head()

,_id,active,createdDate,lastLogin,role,signUpSource,state
0,{'$oid': '5ff1e194b6a9d73a3a9f1052'},True,{'$date': 1609687444800},{'$date': 1609687537858},consumer,Email,WI
1,{'$oid': '5ff1e194b6a9d73a3a9f1052'},True,{'$date': 1609687444800},{'$date': 1609687537858},consumer,Email,WI
2,{'$oid': '5ff1e194b6a9d73a3a9f1052'},True,{'$date': 1609687444800},{'$date': 1609687537858},consumer,Email,WI
3,{'$oid': '5ff1e1eacfcf6c399c274ae6'},True,{'$date': 1609687530554},{'$date': 1609687530597},consumer,Email,WI
4,{'$oid': '5ff1e194b6a9d73a3a9f1052'},True,{'$date': 1609687444800},{'$date': 1609687537858},consumer,Email,WI


In [224]:
users['create_d'] = users['createdDate'].apply(lambda x: extract2(x))
users['login_d'] = users['lastLogin'].apply(lambda x: extract2(x))
users['user_id'] = users['_id'].apply(lambda x: x.get('$oid') if isinstance(x, dict) else None)

In [225]:
users['create_ym'] = users['create_d'].apply(lambda x: datetime.fromtimestamp(x/1000).strftime('%Y-%m') if pd.notnull(x) else None)
users['scan_ym'] = users['login_d'].apply(lambda x: datetime.fromtimestamp(x/1000).strftime('%Y-%m') if pd.notnull(x) else None)

### Missing values
 - Missing values in lastLogin, signUpSource, state and scan_ym columns.

In [226]:
users.shape

(495, 12)

In [227]:
users.isna().sum()

_id              0
active           0
createdDate      0
lastLogin       62
role             0
signUpSource    48
state           56
create_d         0
login_d         62
user_id          0
create_ym        0
scan_ym         62
dtype: int64

### Consistent Values
 - Values are consistent across categorical or text columns

In [228]:
users['signUpSource'].value_counts()

Email     443
Google      4
Name: signUpSource, dtype: int64

In [229]:
users['role'].value_counts()

consumer       413
fetch-staff     82
Name: role, dtype: int64

In [230]:
users['state'].value_counts()

WI    396
NH     20
AL     12
OH      5
IL      3
KY      1
CO      1
SC      1
Name: state, dtype: int64

In [231]:
users['active'].value_counts()

True     494
False      1
Name: active, dtype: int64

### Remove irrelevant columns
 -  Remove json formatted columns and columns that were used for retrieving and conversion.

In [232]:
colsToDrop = ['_id','createdDate','lastLogin']
users.drop(colsToDrop, axis=1, inplace=True)

### Check duplicates
 - Within the users table, 283 users are duplicated in the 495 data points in the table.
 - Drop the 283 duplicated users.

In [233]:
users.duplicated().any()

True

In [234]:
users['user_id'].duplicated().sum()

283

In [235]:
users = users.drop_duplicates()

### Sqlite3 table conversion

In [236]:
for column in users.select_dtypes(include=['float64']).columns:
    users[column] = users[column].fillna(0).astype(int)

In [237]:
users.to_sql('users', conn, if_exists='replace', index=False)

212

In [238]:
file_name = 'users.xlsx'
users.to_excel(file_name, index=False)

### **Business questions - TOP 5 brands**

- What are the top 5 brands by receipts scanned for most recent month?
  - The most recent month in the receipts table is March, 2021.
  - However, due to missing barcode values in receipt table or non listed barcode values in the brand table, most of them have null values. Receipts scanned in Jan, 2021 partially contain brand name values.
  
- How does the ranking of the top 5 brands by receipts scanned for the recent month compare to the ranking for the previous month?
  - Top 5 brands for Jan 2021 are as below (containing name values)
    1. Tostitos
    2. Swanson
    3. Cracker Barrel Cheese
    4. Prego
    5. Diet Chris Cola


In [189]:
cur.execute("""
SELECT scan_ym,
       COUNT(receipt_id)
FROM receipts
GROUP BY scan_ym
ORDER BY scan_ym DESC
""")

rows = cur.fetchall()
for row in rows:
    print(row)

('2021-03', 43)
('2021-02', 500)
('2021-01', 3043)
('2020-11', 24)
('2020-10', 6)


In [196]:
cur.execute("""
SELECT
       A.scan_ym,
       B.name,
       COUNT(A.receipt_id) AS scanned,
       ROW_NUMBER() OVER (PARTITION BY scan_ym ORDER BY COUNT(DISTINCT A.receipt_id) DESC) AS RANK
FROM RECEIPTS A LEFT OUTER JOIN BRANDS B
ON A.barcode=B.barcode
GROUP BY A.scan_ym, B.name
ORDER BY scan_ym DESC
""")

rows = cur.fetchall()
for row in rows:
    print(row)

('2021-03', None, 43, 1)
('2021-02', None, 500, 1)
('2021-01', None, 2961, 1)
('2021-01', 'Tostitos', 23, 2)
('2021-01', 'Swanson', 11, 3)
('2021-01', 'Cracker Barrel Cheese', 10, 4)
('2021-01', 'Prego', 7, 5)
('2021-01', 'Diet Chris Cola', 7, 6)
('2021-01', 'Quaker', 3, 7)
('2021-01', 'Kraft', 3, 8)
('2021-01', 'Kettle Brand', 3, 9)
('2021-01', 'Jell-O', 3, 10)
('2021-01', 'V8', 4, 11)
('2021-01', 'Rice A Roni', 3, 12)
('2021-01', 'Pepperidge Farm', 5, 13)
('2021-01', 'Cheetos', 3, 14)
('2021-01', 'Sargento® Cheese', 1, 15)
('2021-01', 'Pacific Foods', 1, 16)
('2021-01', 'Mountain Dew', 1, 17)
('2021-01', 'Grey Poupon', 1, 18)
('2020-11', None, 24, 1)
('2020-10', None, 6, 1)


###**Business Questions - Spending and items purchased by rewardsReceiptStatus**
- When considering average spend from receipts with 'rewardsReceiptStatus’ of ‘Accepted’ or ‘Rejected’, which is greater?
  - First of all, there is no value named 'Accepted'.
  - If comparing with 'Finished' status, average spending is 913 dollars which is approx. 34 times higher than 'Rejected' average spending of 27 dollars. (Assuming spending in 1 dollar unit.)
- When considering total number of items purchased from receipts with 'rewardsReceiptStatus’ of ‘Accepted’ or ‘Rejected’, which is greater?
  - If comparing with 'Finished' status, total number of items purchased is 491,828 which is approx. 1,927 times higher than 'Rejected' total number of items purchased of 255.


In [208]:
cur.execute("""

    SELECT
        rewardsReceiptStatus,
        ROUND(AVG(CASE WHEN totalSpent IS NOT NULL THEN totalSpent END), 0) AS avg_totalSpent,
        SUM(CASE WHEN purchasedItemCount IS NOT NULL THEN purchasedItemCount ELSE 0 END) AS total_purchasedItemCount
    FROM
        receipts
    GROUP BY
        rewardsReceiptStatus
""")

rows = cur.fetchall()
for row in rows:
    print(row)

('FINISHED', 913.0, 491828)
('FLAGGED', 2064.0, 70063)
('PENDING', 27.0, 0)
('REJECTED', 27.0, 255)
('SUBMITTED', 0.0, 0)


In [216]:
cur.execute("""

WITH StatusTotals AS (
    SELECT
        rewardsReceiptStatus,
        ROUND(AVG(CASE WHEN totalSpent IS NOT NULL THEN totalSpent END), 0) AS avg_totalSpent,
        SUM(CASE WHEN purchasedItemCount IS NOT NULL THEN purchasedItemCount ELSE 0 END) AS total_purchasedItemCount
    FROM
        receipts
    GROUP BY
        rewardsReceiptStatus
)
SELECT
    st1.rewardsReceiptStatus AS status1,
    st2.rewardsReceiptStatus AS status2,
    ROUND((st1.avg_totalSpent - st2.avg_totalSpent) / st2.avg_totalSpent * 100,0) AS percent_change_avg_totalSpent,
    ROUND((st1.total_purchasedItemCount - st2.total_purchasedItemCount) / st2.total_purchasedItemCount * 100,0) AS percent_change_total_purchasedItemCount
FROM
    StatusTotals st1
JOIN
    StatusTotals st2 ON st1.rewardsReceiptStatus < st2.rewardsReceiptStatus

""")

rows = cur.fetchall()
for row in rows:
    print(row)


('FINISHED', 'FLAGGED', -56.0, 600.0)
('FINISHED', 'PENDING', 3281.0, None)
('FINISHED', 'REJECTED', 3281.0, 192700.0)
('FINISHED', 'SUBMITTED', None, None)
('FLAGGED', 'PENDING', 7544.0, None)
('FLAGGED', 'REJECTED', 7544.0, 27300.0)
('FLAGGED', 'SUBMITTED', None, None)
('PENDING', 'REJECTED', 0.0, -100.0)
('PENDING', 'SUBMITTED', None, None)
('REJECTED', 'SUBMITTED', None, None)


### Business Questions -
- Which brand has the most spend among users who were created within the past 6 months?
    - Tostitos has the most spending of 15,785 dollars among users who were created within the past 6 months.
- Which brand has the most transactions among users who were created within the past 6 months?
    - Tostitos has the most transactions of 23 among users who were created within the past 6 months.

In [244]:
cur.execute("""
SELECT A.name,
       ROUND(SUM(B.totalSpent), 0) AS "total_spend",
       COUNT(B.receipt_id) AS "transaction"
FROM brands A
INNER JOIN receipts B ON A.barcode = B.barcode
INNER JOIN users C ON B.userId = C.user_id
WHERE A.name IS NOT NULL
AND C.create_ym >= '2020-09'
GROUP BY A.name
HAVING ROUND(AVG(B.totalSpent), 0) > 0
ORDER BY "total_spend" DESC;
""")

# Fetch and print the results of the SELECT query
rows = cur.fetchall()
for row in rows:
    print(row)

('Tostitos', 15785.0, 23)
('Pepperidge Farm', 14163.0, 3)
('V8', 9442.0, 2)
('Prego', 9442.0, 2)
('Diet Chris Cola', 9442.0, 2)
('Swanson', 7180.0, 11)
('Cracker Barrel Cheese', 4884.0, 2)
('Jell-O', 4753.0, 2)
('Cheetos', 4721.0, 1)
('Kettle Brand', 2399.0, 3)
('Grey Poupon', 743.0, 1)
('Quaker', 32.0, 1)
